# Sprint アンサンブル学習

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import statistics
import math
import itertools
import copy

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df_train = pd.read_csv("../Sprint/train.csv")
df_train = df_train.set_index("Id")
df_test = pd.read_csv("../Sprint/test.csv")
df_test = df_test.set_index("Id")
# display(df_train.head())
# display(df_test.head())

In [3]:
# (問題文)目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。
# 訓練データと検証データに分割
X1 = df_train.loc[:, ["GrLivArea", "YearBuilt"]].values
y1 = df_train.iloc[:, -1].values

X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=0)
print(X1_train.shape)
print(X1_test.shape)
print(y1_train.shape)
print(y1_test.shape)

(1168, 2)
(292, 2)
(1168,)
(292,)


In [4]:
# 学習関数
# 標準化しないモデル
def _notScaler(X, y, model):
    # (問題文)train.csvを学習用（train）8割、検証用（val）2割に分割
    X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=0)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse

In [5]:
# 学習関数
# 標準化しないモデル
def _Scaler(X, y, model):
    # (問題文)train.csvを学習用（train）8割、検証用（val）2割に分割
    X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=0)
    scaler = StandardScaler()
    scaler.fit(X_train)   #後でスケーリングするために使用する平均と標準を計算します。
    X_train_scaler = scaler.transform(X_train)   # 標準化　センタリングとスケーリングによって標準化を実行する
    X_test_scaler = scaler.transform(X_test)     # 標準化　センタリングとスケーリングによって標準化を実行する
    
    model.fit(X_train_scaler, y_train)
    y_pred = model.predict(X_test_scaler)
    mse = mean_squared_error(y_test, y_pred)

    return mse

In [6]:
# グラフ化関数
# classの実装で利用
def _yyplot(y_test, y_pred, model):
    mse = mean_squared_error(y_test, y_pred)
    yvalues = np.concatenate([y_test.flatten(), y_pred.flatten()])
    ymin, ymax, yrange = np.amin(yvalues), np.amax(yvalues), np.ptp(yvalues)
    fig = plt.figure(figsize=(8, 8))
    plt.scatter(y_test, y_pred, label='MSE : %.3f, MAE : %.3f \n R2 = %.3f'%(
                mse, mae, r2s))
    
    plt.plot([ymin - yrange * 0.01, ymax + yrange * 0.01], [ymin - yrange * 0.01, ymax + yrange * 0.01])
    plt.xlim(ymin - yrange * 0.01, ymax + yrange * 0.01)
    plt.ylim(ymin - yrange * 0.01, ymax + yrange * 0.01)
    plt.xlabel('y_test', fontsize=24)
    plt.ylabel('y_predicted', fontsize=24)
    plt.title('Plot by {}'.format(model), fontsize=16)
    plt.tick_params(labelsize=16)
    plt.legend()
    plt.show()

    return fig

In [7]:
# 線形回帰モデル用意
lr = LinearRegression()
svm = SVR()
dt = DecisionTreeRegressor(random_state=0)
dt2 = DecisionTreeRegressor(max_depth=10, max_leaf_nodes=5, random_state=0)
reg = linear_model.Lasso(alpha=0.1)
NN = MLPRegressor(random_state=0, max_iter=50)

In [8]:
# 単独学習
index_list = ["MSE"]
model_dict = {"LinearRegression": lr, "SVM": svm, 
              "DecisionTreeRegressor": dt, "DecisionTreeRegressor(Param_change)": dt2,
              "Lasso": reg, "NeuralN": NN}
df_table = pd.DataFrame(np.zeros, index=index_list, columns=model_dict.keys())

for model_name, model in model_dict.items():
    df_table[model_name] = _notScaler(X1, y1, model=model)

df_table

/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,LinearRegression,SVM,DecisionTreeRegressor,DecisionTreeRegressor(Param_change),Lasso,NeuralN
MSE,2.942067e+09,7.243320e+09,3.009170e+09,2.916621e+09,2.942067e+09,2.840032e+10


In [9]:
# 単独学習
# 標準化
index_list = ["MSE"]
model_dict = {"LinearRegression": lr, "SVM": svm, 
              "DecisionTreeRegressor": dt, "DecisionTreeRegressor(Param_change)": dt2,
              "Lasso": reg, "NeuralN": NN}

df_table_scaler = pd.DataFrame(np.zeros, index=index_list, columns=model_dict.keys())

for model_name, model in model_dict.items():
    df_table_scaler[model_name] = _Scaler(X1, y1, model=model)
    
df_table_scaler

/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,LinearRegression,SVM,DecisionTreeRegressor,DecisionTreeRegressor(Param_change),Lasso,NeuralN
MSE,2.942067e+09,7.221625e+09,2.983519e+09,2.916621e+09,2.942065e+09,3.979373e+10


## 単一モデルのMSEの値が確認できた
- 線形回帰モデルが一番低い値となった。
- 標準化処置の有無で若干の値変動はあるが、そこまで重要でない模様。
- 線形回帰の2.942e+9を基準とする。

# 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。


ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation

In [10]:
class ScratchBlending():
    """
    K-meansのスクラッチ実装

    Parameters
    ----------  ##########TODOあとで書き換える###########
    estimators : 学習モデル群_辞書型
    n_jobs : 使うか不明だが仮置き
    weights :使うか不明だが仮置き
    verbose : 学習過程のプリント
    """
    def __init__(self, estimators, n_jobs=None, weights=None, verbose=False):
        # ハイパーパラメータを属性として記録
        self.estimators = estimators
        self.n_jobs = n_jobs
        self.weights = weights
        self.verbose = verbose

    
    def fit(self, X, y):
        """
        複数の学習モデルをfor文で全通り学習させる
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データのラベル
        """
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        self.fit_data_list = []
        
        for model in self.estimators:
            fit_data = model.fit(X, y)
            self.fit_data_list.append(fit_data)           
            
        if self.verbose:
            pass
            #verboseをTrueにした際は学習過程を出力

    def predict(self, X):
        """
        K-meansによるクラスタリングを計算
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        
        return
        pred_values_mean : 次の形のndarray, shape (n_samples, )
        訓練データの学習モデル分のpredict値の平均

        """
        # 平均値を取る箱を作成
        pred_values = np.zeros((len(self.estimators), X.shape[0]))
        
        # 全モデルのpredictメソッドをループさせる
        for i, model in enumerate(self.fit_data_list):
            pred_values[i] = model.predict(X)
        
        # 平均値を計算
        pred_values_mean = np.mean(pred_values, axis=0)
        
        return pred_values_mean

In [11]:
model_list = [lr, svm, dt, dt2, reg, NN]

## 例1（学習モデルは3つ、特徴量は未標準化）

In [12]:
# 学習モデル3つ(線形回帰, SVM, 決定木)
# 未標準化
model1_list = model_list[:3]

model1 = ScratchBlending(estimators=model1_list)

# 未標準化の訓練データで学習
model1.fit(X1_train, y1_train)

# 予測値は全モデルの平均値
pred1 = model1.predict(X1_test)

# MSEを算出
mse = mean_squared_error(y1_test, pred1)

print("[model1]")
print("MSE:{:.3e}".format(mse))

[model1]
MSE:2.722e+09


### 線形回帰と決定木の単独モデルより少し高い結果になった。

## 例2（学習モデルは3つ、特徴量を標準化）

In [13]:
# 標準化
scaler = StandardScaler()

scaler.fit(X1_train)   #後でスケーリングするために使用する平均と標準を計算します。
X1_train_scaler = scaler.transform(X1_train)   # 標準化　センタリングとスケーリングによって標準化を実行する
X1_test_scaler = scaler.transform(X1_test)     # 標準化　センタリングとスケーリングによって標準化を実行する

# 学習モデルはmodel1と同様
model2 = ScratchBlending(estimators=model1_list)

# 標準化の訓練データで学習
model2.fit(X1_train_scaler, y1_train)

# 予測値は全モデルの平均値
pred2 = model2.predict(X1_test_scaler)

# MSEを算出
mse2 = mean_squared_error(y1_test, pred2)

print("[model2]")
print("MSE:{:.3e}".format(mse2))

[model2]
MSE:2.719e+09


/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


### 例2も下がるには下がったが、単独で最も数値が悪かったSVMが入っているのは悪影響？
- 標準化の有無はさほど影響なしと思われる

## 例3　SVMを含めない3つのモデルでブレンディング

In [14]:
# 学習モデル3つ(決定木パラム変更, Lasso, NN)
model3_list = model_list[-3:]

model3 = ScratchBlending(estimators=model3_list)

# 未標準化の訓練データで学習
model3.fit(X1_train, y1_train)

# 予測値は全モデルの平均値
pred3 = model3.predict(X1_test)

# MSEを算出
mse3 = mean_squared_error(y1_test, pred3)

print("[model3]")
print("MSE:{:.3e}".format(mse3))

[model3]
MSE:5.642e+09


/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


### ものすごく悪くなった。。。原因不明。

## 例4~6（学習モデルは2つ_各組み合わせで実行）

In [15]:
# 学習モデル組み合わせ
model_list4 = model_list[:2]       # 線形回帰とSVMの2つ
model_list5 = model_list[3:4]    # 決定木（パラムチェンジ）とlasso
model_list6 = model_list[-2:]     # lassoとNN

model4 = ScratchBlending(estimators=model_list4)
model5 = ScratchBlending(estimators=model_list5)
model6 = ScratchBlending(estimators=model_list6)

# 標準化の訓練データで学習
model4.fit(X1_train, y1_train)
model5.fit(X1_train, y1_train)
model6.fit(X1_train, y1_train)

# 予測値は全モデルの平均値
pred4 = model4.predict(X1_test)
pred5 = model5.predict(X1_test)
pred6 = model6.predict(X1_test)

# MSEを算出
mse4 = mean_squared_error(y1_test, pred4)
mse5 = mean_squared_error(y1_test, pred5)
mse6 = mean_squared_error(y1_test, pred6)

print("[学習モデル:線形回帰とSVM]MSE:{:.3e}".format(mse4))
print("[決定木（パラムチェンジ）とlasso]MSE:{:.3e}".format(mse5))
print("[lassoとNN]MSE:{:.3e}".format(mse6))

[学習モデル:線形回帰とSVM]MSE:3.776e+09
[決定木（パラムチェンジ）とlasso]MSE:2.917e+09
[lassoとNN]MSE:9.107e+09


/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


### 2つの組み合わせだとさほど変わらない、悪くなることもある

## 例7~9　学習モデルを4つ以上で複数組み合わせ

In [16]:
# 例7−9
model_list7 = model_list[:4]    # 線形回帰とSVMの2つ
model_list8 = model_list[:5]    # 決定木（パラムチェンジ）とlasso
model_list9 = model_list[:]      # lassoとNN

model7 = ScratchBlending(estimators=model_list7)
model8 = ScratchBlending(estimators=model_list8)
model9 = ScratchBlending(estimators=model_list9)

# 標準化の訓練データで学習
model7.fit(X1_train, y1_train)
model8.fit(X1_train, y1_train)
model9.fit(X1_train, y1_train)

# 予測値は全モデルの平均値
pred7 = model7.predict(X1_test)
pred8 = model8.predict(X1_test)
pred9 = model9.predict(X1_test)

# MSEを算出
mse7 = mean_squared_error(y1_test, pred7)
mse8 = mean_squared_error(y1_test, pred8)
mse9 = mean_squared_error(y1_test, pred9)

print("[学習モデル4つ]MSE:{:.3e}".format(mse7))
print("[学習モデル5つ]MSE:{:.3e}".format(mse8))
print("[学習モデル6つ]MSE:{:.3e}".format(mse9))

[学習モデル4つ]MSE:2.576e+09
[学習モデル5つ]MSE:2.523e+09
[学習モデル6つ]MSE:3.608e+09


/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## MSEが小さくなったモデルは例1と例7と例8のモデル（例2は参考外）

In [17]:
print("基準値MSEは線形回帰単独モデルの2.942e+9")
print("[model1:学習モデル3つ]MSE:{:.3e}".format(mse))
print("[model7:学習モデル4つ]MSE:{:.3e}".format(mse7))
print("[model8:学習モデル5つ]MSE:{:.3e}".format(mse8))

基準値MSEは線形回帰単独モデルの2.942e+9
[model1:学習モデル3つ]MSE:2.722e+09
[model7:学習モデル4つ]MSE:2.576e+09
[model8:学習モデル5つ]MSE:2.523e+09


## 最終考察
- ブレンディングの効果について理解できた。
- NeuralNetworkモデルが入ったケースは悪くなった印象
- 学習モデルを増やせばMSEも下がる傾向だが、model9のように悪影響を与えてしまうケースもあることがわかった。

## 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。



In [18]:
class ScratchBagging():
    """
    K-meansのスクラッチ実装

    Parameters
    ----------  ##########TODOあとで書き換える###########
    estimators : 学習モデル群_辞書型
    n_jobs : 
    weights :
    verbose : 
    """
    def __init__(self, model, bagging, n_jobs=None, weights=None, verbose=False):
        # ハイパーパラメータを属性として記録
        self.model = model
        self.bagging = bagging
        self.n_jobs = n_jobs
        self.weights = weights
        self.verbose = verbose

    
    def fit(self, X, y):
        """
        K-meansによるクラスタリングを計算
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データのラベル
        """
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        self.fit_data_list = []
        
        for i in range(self.bagging):
            _, bootstrap_X, _ , bootstrap_y  = train_test_split(X, y, shuffle=True, test_size=0.2)
            model = copy.deepcopy(self.model)
            fit_data = model.fit(bootstrap_X, bootstrap_y)
            self.fit_data_list.append(fit_data)
            
                
        if self.verbose:
            pass
            #verboseをTrueにした際は学習過程を出力

    def predict(self, X):
        """
        K-meansによるクラスタリングを計算
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        
        return
        pred_values_mean : 次の形のndarray, shape (n_samples, )
        訓練データの学習モデル分のpredict値の平均

        """
        # 平均値を取る箱を作成
        pred_values = np.zeros((self.bagging, X.shape[0]))
        
        # 全モデルのpredictメソッドをループさせる
        for i, model in enumerate(self.fit_data_list):
            pred_values[i] = model.predict(X)
                
        # 平均値を計算
        pred_values_mean = np.mean(pred_values, axis=0)
        
        return pred_values_mean

In [19]:
scr_bagging = ScratchBagging(model=lr,bagging=10, verbose=False)
scr_bagging.fit(X1_train, y1_train)
bag_pred = scr_bagging.predict(X1_test)

In [20]:
mse10 = mean_squared_error(y1_test, bag_pred)

print("[学習モデル:線形回帰_バギング:10model]MSE:{:.3e}".format(mse10))

[学習モデル:線形回帰_バギング:10model]MSE:2.965e+09


### 単独モデルより若干下がった・・・

### 【自習】決定木モデルで仮想ランダムフォレスト

In [21]:
scr_bagging2 = ScratchBagging(model=dt,bagging=10, verbose=False)

scr_bagging2.fit(X1_train, y1_train)

bag_pred2 = scr_bagging2.predict(X1_test)

In [22]:
mse10 = mean_squared_error(y1_test, bag_pred2)
print("[学習モデル:決定木_バギング:10model]MSE:{:.3e}".format(mse10))

[学習モデル:決定木_バギング:10model]MSE:2.002e+09


### 学習モデルを決定木に変えたところ、大きく減少した。

## 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは $K_0=3, M_0=2$ 程度にします。


《学習時》


（ステージ $0$ ）


学習データを $K_0$ 個に分割する。
分割した内の $(K_0 - 1)$ 個をまとめて学習用データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。
あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、 $M_0$ 個のブレンドデータが得られる。

（ステージ $n$ ）


ステージ $n-1$ のブレンドデータを$M_{n-1}$ 次元の特徴量を持つ学習用データと考え、 $K_n$ 個に分割する。以下同様である。

（ステージ $N$ ）＊最後のステージ


ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ $0$ ）


テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$ ）


ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$ ）＊最後のステージ


ステージ $N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [23]:
class ScratchStacking():
    """
    スタッキング

    Parameters
    ----------

    """
    def __init__(self, k=3, verbose=False, verbose_interval=1, random_state=None):
        # ハイパーパラメータを属性として記録
        self.k = k
        self.verbose = verbose
        self.verbose_interval = verbose_interval
        self.random_state = random_state
        self.next_stage = None
       
    def fit(self, X, y):
        """
        
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        """
        
        self.n_samples = len(X)
        self.n_models = 3
        self.blend_data = np.zeros([self.n_samples, self.n_models])
        # 学習
        self.trainingForStacking = TrainingForStacking(self.n_samples, self.n_models, self.k, self.verbose, self.verbose_interval, self.random_state)
        self.blend_data = self.trainingForStacking._stage_n(X, y, self.blend_data)
        # 学習（ラストステージ）
        self.trainingForStacking._last_stage(self.blend_data, y)
        
        #verboseをTrueにした際は学習過程を出力            

    def predict(self, X):
        """
        推定
        """
        preds = np.zeros([len(X), self.n_models])
        pre_pred = self.trainingForStacking._predict_stage_n(X, preds) / self.n_samples
        pred = self.trainingForStacking._predict_last_stage(pre_pred)
        
        return pred

In [24]:
class TrainingForStacking():
    def __init__(self, n_samples, n_models, k, verbose, verbose_interval, random_state):
        # ハイパーパラメータを属性として記録
        self.k = k
        self.verbose = verbose
        self.verbose_interval = verbose_interval
        self.random_state = random_state
        self.next_stage = None
        self.model_list = ['LinearRegression',
                           'DecisionTreeRegressor',
                           "Lasso"]
        self.n_samples = n_samples
        self.n_models = n_models
        self.all_idx = np.arange(self.n_samples)
        self.div_idx = np.array_split(self.all_idx, self.k)
        
        self.stop = False
        
    def _stage_n(self, X, y, blend_data, div_no=0):
        self.div_no = div_no
        self.blend_data = blend_data.copy()
        
        self.other_idx = np.delete(self.all_idx, self.div_idx[self.div_no])
        
        X_train, X_test = X[self.other_idx, :], X[self.div_idx[self.div_no]]
        y_train, y_test = y[self.other_idx], y[self.div_idx[self.div_no]]
        
        self.model_0 = LinearRegression()
        self.model_1 = DecisionTreeRegressor()
        self.model_2 = linear_model.Lasso(alpha=0.1)

        self.model_0.fit(X_train, y_train)
        self.model_1.fit(X_train, y_train)
        self.model_2.fit(X_train, y_train)
        
        self.blend_data[self.div_idx[self.div_no], 0] = self.model_0.predict(X_test)
        self.blend_data[self.div_idx[self.div_no], 1] = self.model_1.predict(X_test)
        self.blend_data[self.div_idx[self.div_no], 2] = self.model_2.predict(X_test)
        
        self.mse_0 = mean_squared_error(y_test, self.blend_data[self.div_idx[self.div_no], 0])
        self.mse_1 = mean_squared_error(y_test, self.blend_data[self.div_idx[self.div_no], 1])
        self.mse_2 = mean_squared_error(y_test, self.blend_data[self.div_idx[self.div_no], 2])

        if self.verbose:
            print('**********', self.model_list[0], '-', self.div_no, '**********')
            print('MSE(平均二乗誤差)(test) : {:,.3e}'.format(self.mse_0))
            print('**********', self.model_list[1], '-', self.div_no, '**********')
            print('MSE(平均二乗誤差)(test) : {:,.3e}'.format(self.mse_1))
            print('**********', self.model_list[2], '-', self.div_no, '**********')
            print('MSE(平均二乗誤差)(test) : {:,.3e}'.format(self.mse_2))
        
        self.div_no += 1
        if self.div_no >= self.k:
            self.stop = True
            return self.blend_data
        self.next_stage = TrainingForStacking(self.n_samples, self.n_models, self.k, self.verbose, self.verbose_interval, self.random_state)
        self.blend_data = self.next_stage._stage_n(X, y, self.blend_data, self.div_no)
        
        return self.blend_data

        
        
    def _last_stage(self, blend_data, y):
        X_train, X_test, y_train, y_test = train_test_split(blend_data, y, test_size=0.2, random_state=0)
        self.model_last = DecisionTreeRegressor()
        self.model_last.fit(X_train, y_train)
        last_pred_train = self.model_last.predict(X_train)
        last_pred = self.model_last.predict(X_test)
        
        return last_pred
        l
    def _predict_stage_n(self, X, preds):
        if self.stop:
            return preds
        else:
            preds[:, 0] += self.model_0.predict(X)
            preds[:, 1] += self.model_1.predict(X)
            preds[:, 2] += self.model_2.predict(X)
            return self.next_stage._predict_stage_n(X, preds)
        
    def _predict_last_stage(self, X):
        pred = self.model_last.predict(X)
        return pred


In [25]:
scr_st = ScratchStacking(verbose=True, random_state=0)
scr_st.fit(X1_train, y1_train)

st_pred = scr_st.predict(X1_test)

mse11 = mean_squared_error(y1_test, st_pred)
print("[学習モデル:決定木_バギング:10model]MSE:{:.3e}".format(mse11))

********** LinearRegression - 0 **********
MSE(平均二乗誤差)(test) : 2.156e+09
********** DecisionTreeRegressor - 0 **********
MSE(平均二乗誤差)(test) : 3.405e+09
********** Lasso - 0 **********
MSE(平均二乗誤差)(test) : 2.156e+09
********** LinearRegression - 1 **********
MSE(平均二乗誤差)(test) : 2.378e+09
********** DecisionTreeRegressor - 1 **********
MSE(平均二乗誤差)(test) : 3.469e+09
********** Lasso - 1 **********
MSE(平均二乗誤差)(test) : 2.378e+09
********** LinearRegression - 2 **********
MSE(平均二乗誤差)(test) : 1.600e+09
********** DecisionTreeRegressor - 2 **********
MSE(平均二乗誤差)(test) : 2.115e+09
********** Lasso - 2 **********
MSE(平均二乗誤差)(test) : 1.600e+09
[学習モデル:決定木_バギング:10model]MSE:2.164e+09


### スタッキングでもMSEが下がった。